# АНАЛИЗ БАЗЫ ДАННЫХ СЕРВИСА ДЛЯ ЧТЕНИЯ КНИГ

## Цели исследования 
Заказчиком выступает компания, купившая сервис для чтения книг по подписке. Необходимо проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

От заказчика поступили следующие требования:

* Посчитать, сколько книг вышло после 1 января 2000 года;
* Для каждой книги посчитать количество обзоров и среднюю оценку;
* Определить издательство, которое выпустило наибольшее число книг толще 50 страниц, чтобы исключить из анализа брошюры;
* Определить автора с самой высокой средней оценкой книг, при этом учитывать только книги с 50 и более оценками;
* Посчитать среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## Схема данных

![Image of Yaktocat](https://concrete-web-bad.notion.site/image/https%3A%2F%2Fs3-us-west-2.amazonaws.com%2Fsecure.notion-static.com%2F069818d1-0e5c-4d87-a461-0de584ab9c33%2FUntitled_(33).png?id=bd53c8db-b4fd-49eb-8cc9-572ebb3c9163&table=block&spaceId=9e4bd47b-c6e6-4ca3-bcee-279794b47315&width=2000&userId=&cache=v2)

## Загрузка и подготовка данных

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
import datetime as dt

In [2]:
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# функция для запроса данных
def request(query):
    df = pd.io.sql.read_sql(query, con = engine) 
    df.info()
    display(df.head(10))
    return df

In [4]:
# получаем сырые данные БД books
query = '''
            SELECT *
            FROM books
        '''
df = request(query)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   book_id           1000 non-null   int64 
 1   author_id         1000 non-null   int64 
 2   title             1000 non-null   object
 3   num_pages         1000 non-null   int64 
 4   publication_date  1000 non-null   object
 5   publisher_id      1000 non-null   int64 
dtypes: int64(4), object(2)
memory usage: 47.0+ KB


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


In [5]:
# получаем сырые данные БД authors
query = '''
            SELECT *
            FROM authors
        '''
authors = request(query)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 636 entries, 0 to 635
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   author_id  636 non-null    int64 
 1   author     636 non-null    object
dtypes: int64(1), object(1)
memory usage: 10.1+ KB


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


In [6]:
# получаем сырые данные БД publishers
query = '''
            SELECT *
            FROM publishers
        '''
authors = request(query)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   publisher_id  340 non-null    int64 
 1   publisher     340 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.4+ KB


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


In [7]:
# получаем сырые данные БД ratings
query = '''
            SELECT *
            FROM ratings
        '''
authors = request(query)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6456 entries, 0 to 6455
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   rating_id  6456 non-null   int64 
 1   book_id    6456 non-null   int64 
 2   username   6456 non-null   object
 3   rating     6456 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 201.9+ KB


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


In [8]:
# получаем сырые данные БД reviews
query = '''
            SELECT *
            FROM reviews
        '''
authors = request(query)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2793 entries, 0 to 2792
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review_id  2793 non-null   int64 
 1   book_id    2793 non-null   int64 
 2   username   2793 non-null   object
 3   text       2793 non-null   object
dtypes: int64(2), object(2)
memory usage: 87.4+ KB


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With pe...
7,8,4,abbottjames,Within enough mother. There at system full rec...
8,9,5,npowers,Thank now focus realize economy focus fly. Ite...
9,10,5,staylor,Game push lot reduce where remember. Including...


## Описание данных
**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

## Подготовка запросов

In [9]:
# функция для запроса данных
def request(query):
    df = pd.io.sql.read_sql(query, con = engine) 
    display(df.head(len(df)))
    return df

### Cколько книг вышло после 1 января 2000 года

In [10]:
query = '''
SELECT COUNT(book_id) 
FROM books
WHERE publication_date >= '2000-01-01'
'''

request_1 = request(query)

,count
0,821


Всего в базе данных содержится 821 книга, вышедшая после 1 января 2000 года.

### Количество обзоров и среднюю оценку для каждой книги

In [11]:
query = """
SELECT title,
       COUNT(DISTINCT review_id) AS review_count,
       ROUND(AVG(rating), 2) AS rating_avg
FROM books AS b 
     LEFT JOIN ratings AS r ON b.book_id = r.book_id
     LEFT JOIN reviews AS rev ON b.book_id = rev.book_id
GROUP BY b.book_id
ORDER BY rating_avg DESC, review_count DESC
"""

request_2 = request(query)

,title,review_count,rating_avg
0,A Dirty Job (Grim Reaper #1),4,5.00
1,School's Out—Forever (Maximum Ride #2),3,5.00
2,Moneyball: The Art of Winning an Unfair Game,3,5.00
3,The Big Bad Wolf (Alex Cross #9),2,5.00
4,Welcome to Temptation (Dempseys #1),2,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3,2.25
996,Drowning Ruth,3,2.00
997,His Excellency: George Washington,2,2.00
998,Junky,2,2.00


Мы получили таблицу - список книг, отсортированных по убыванию количества обзоров и среднего рейтинга. Возглавляет его роман американского писателя Кристофера Мура.

### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [12]:
query = '''
SELECT 
    publishers.publisher,
    CASE
        WHEN books.books_num IS NULL THEN 0
        ELSE books.books_num
    END AS books_num
FROM publishers
LEFT JOIN 
    (SELECT publisher_id, 
            COUNT(book_id) books_num
    FROM books
    WHERE num_pages > 50
    GROUP BY publisher_id
    ) AS books ON publishers.publisher_id = books.publisher_id
ORDER BY books_num DESC
LIMIT 5
'''

request_3 = request(query)

,publisher,books_num
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


Лидирующим издательством является Penguin Books.

### Автор с самой высокой средней оценкой книг
Учитываются только книги с 50 и более оценками

In [13]:
query = '''
SELECT authors.author,
       AVG(avg_rating) avg_rating
    
FROM
    books
INNER JOIN 
    (SELECT book_id,
            COUNT(rating_id) ratings_num,
            AVG(rating) avg_rating
    FROM ratings
    GROUP BY book_id 
    HAVING COUNT(rating_id) >= 50
    ) AS subq_b ON subq_b.book_id = books.book_id
INNER JOIN authors ON authors.author_id = books.author_id
GROUP BY authors.author_id, authors.author
ORDER BY avg_rating DESC
LIMIT 5
'''

request_4 = request(query)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645


Наивысшая оценка принадлежит британской писатильнеце Джоан Роулинг, автору серии книг о Гарри Поттере.

### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
query = '''
SELECT AVG(reviews_num)
FROM
    (SELECT reviews.username,
            COUNT(*) reviews_num
    FROM reviews
    INNER JOIN 
        (SELECT username
        FROM ratings
        GROUP BY username 
        HAVING COUNT(rating_id) > 50
        ) AS users ON users.username = reviews.username
    GROUP BY reviews.username
    ) rev
'''
request_5 = request(query)

,avg
0,24.333333


Среднее количество обзоров для активных пользователей: 24.

## Вывод

1. Всего в базе данных содержится 821 книга, вышедшая после 1 января 2000 года. Это примерно 82%.
2. Мы получили таблицу - список книг, отсортированных по убыванию количества обзоров и среднего рейтинга. Возглавляет его роман американского писателя Кристофера Мура.
3. Лидирующей тройкой издательств является Penguin Books, Vintage и	Grand Central Publishing.
4. Наивысшая оценка принадлежит британской писатильнеце Джоан Роулинг, автору серии книг о Гарри Поттере.
5. Среднее количество обзоров для активных пользователей: 24.